In [ ]:
spark_version = "3.5.2"
scala_version = "2.12"

packages = [
    f"org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}",
    "org.apache.kafka:kafka-clients:3.8.0",
]

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = (
    SparkSession.builder.master("local")
    .appName("dataStreamingApp")
    .config("spark.jars.packages", ",".join(packages))
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")
spark

In [ ]:
!spark-submit --class dataStreamingApp --packages org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.2 spark-structured-streaming.ipynb

In [ ]:
df = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("subscribe", "user_entry")
    .option("startingOffsets", "earliest")
    .load()
)

In [ ]:
df = df.withColumn("data", F.col("value").cast("string"))

In [ ]:
df

In [ ]:
df.select("data").writeStream.format("console").start()